In [8]:
import pandas as pd #package for reading data
import numpy as np
import graphlib
import statistics
import matplotlib.pyplot as plt #package for plots
import gmplot 
from scipy import signal

#main function
def der(x,y):
  """
  derivative of column y of sheet x
  """
  i = 0
  time = y
  #value = reduce_bar
  value = x
  slopes = []
  aslopes = []
  times = []
  while i < len(x)-1: 
    x0, x1 = time.iat[i], time.iat[i+1]
    t = time.iat[i]
    y0, y1 = value[i], value[i+1]
    numerator = y1-y0
    num = numerator
    if numerator < 0 :  # somehow abs() doesn't work
      numerator *= -1
    slope = numerator/(x1-x0)
    #slope = numerator
    aslope = num /(x1-x0)
    #aslope = num
    slopes.append(slope)
    times.append([x0,x1])
    aslopes.append(aslope)
    i+=1
  slopes.append(0)
  times.append(0)
  aslopes.append(0)
  return [slopes,times,aslopes]

gps_data = pd.read_csv('GPS.csv')
pressure_data = pd.read_csv('Pressure.csv')
pressure_range = pd.read_csv("Pressure_Range.csv")
acceleration_data = pd.read_csv("Acceleration.csv")

latitude = gps_data["Latitude"]
longitude = gps_data["Longitude"]
coordinates = [latitude,longitude]
gps_time = gps_data["time"]
altitude = gps_data["Altitude"]

bar = pressure_data["Pressure"]
bar_time = pressure_data["time"]
reduce_bar = signal.savgol_filter(bar,window_length=11, polyorder=1, mode="nearest")

a_x = acceleration_data["Acceleration x"]
az =acceleration_data["Acceleration z"]
ay = acceleration_data["Acceleration y"]
at = acceleration_data["time"]

print(len(gps_time))
print(len(bar_time))

new_gps_time = np.round(gps_time)
new_bar_time = np.round(bar_time)

deri = der(bar,new_bar_time)
der_p = deri[0]
der_t = deri[1]
der_a = deri[2]

std = statistics.stdev(der_p)
range =  der_p[0] + std

new_t = []
new_a = []
for n in der_p:
    if n >= range:
        new_t.append(der_t[der_p.index(n)])
        new_a.append(der_a[der_p.index(n)])



# for n in coordinates:
#     if (new_t[i] == new_gps_time[i]) and (new_a[i] > 0) :
#         new_coordinates_up.append(n)
#     elif (new_t[i] == new_gps_time[i]) and (new_a[i] < 0) :
#         new_coordinates_down.append(n)
#     elif  (new_t[i] == new_gps_time[i]) and (new_a[i] == 0):
#         new_coordinates_flat.append(n)
#     i = i + 1

up_time = []
down_time = []
flat_time = []
for n in new_a:
  if n < 0:
    up_time.append(new_t[new_a.index(n)])
  elif n > 0 :
    down_time.append(new_t[new_a.index(n)])
  else:
    flat_time.append(new_t[new_a.index(n)])

def gather_function(time_data):
  data1 = []
  data2 = []
  i = 0
  x = 0
  while i < len(time_data) - 1:
    for n in gps_time:
      if(time_data[i] <= n <=time_data[i+1]):
        data1.append(latitude[x])
        data2.append(longitude[x])
        x = x + 1
    i = i + 1 
  return [data1,data2]

new_coordinates_up = gather_function(up_time)
new_coordinates_down = gather_function(down_time)
new_coordinates_flat = gather_function(flat_time)


lat = latitude[0]
log = longitude[0]

print(new_coordinates_down)

gmap = gmplot.GoogleMapPlotter(lat, log, zoom=18)
gmap.plot(latitude, longitude, '#771c2f', size = 0.5, marker = False )
gmap.marker(lat,log, color='cornflowerblue')
# gmap.scatter(gps_hill_la, gps_hill_lo, '#16EBF9', size = 0.5, marker = False  )
# gmap.scatter(new_coordinates_up, up_lo, '#FFFF00', size = 0.5, marker = False  )
# gmap.scatter(down_la, down_lo, '#16EBF9', size = 0.5, marker = False  )
# gmap.plot(latitude, longitude, '#771c2f', size = 0.5, marker = False )





99
98


TypeError: '<=' not supported between instances of 'list' and 'float'